# Date: March 10, Total mass balance using water content

In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.append('/Users/ajc/CODES/ats/tools/utils/')
import get_transect_data

In [3]:
varnames = ['water_content','molar_density_liquid']
Data_B = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile,varnames[0])
Data_B[varnames[0]] = np.array(Data)
Time, Data = get_transect_data.get2DMatrix(infile,varnames[1])
Data_B[varnames[1]] = np.array(Data)
Data_B['time'] = np.array(Time)

In [4]:
Data_S30 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S30_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile,varnames[0])
Data_S30[varnames[0]] = np.array(Data)
Time, Data = get_transect_data.get2DMatrix(infile,varnames[1])
Data_S30[varnames[1]] = np.array(Data)
Data_S30['time'] = np.array(Time)

In [5]:
Data_S60 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S60_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile,varnames[0])
Data_S60[varnames[0]] = np.array(Data)
Time, Data = get_transect_data.get2DMatrix(infile,varnames[1])
Data_S60[varnames[1]] = np.array(Data)
Data_S60['time'] = np.array(Time)

In [6]:
Data_S90 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile,varnames[0])
Data_S90[varnames[0]] = np.array(Data)
Time, Data = get_transect_data.get2DMatrix(infile,varnames[1])
Data_S90[varnames[1]] = np.array(Data)
Data_S90['time'] = np.array(Time)

In [30]:
# return total water content in the system
def get_water_content(season = " "):
    #25 * 2cm = 50 cm
    data = []
    dataDensity = []
    times = []
    if season == "B":
        data = Data_B[varnames[0]]
        dataDensity = Data_B[varnames[1]]
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30[varnames[0]]
        dataDensity = Data_S30[varnames[1]]
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60[varnames[0]]
        dataDensity = Data_S60[varnames[1]]
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90[varnames[0]]
        dataDensity = Data_S90[varnames[1]]
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT = dict()
    
    AvgTemp = np.zeros(cycles)
    MaxTemp = np.zeros(cycles)
    MinTemp = np.zeros(cycles)
    #print depth
    #depth_cm = data[1,0,0,80] - data[1,0,0,80-depth]
    #print (season, start, end, depth_s, depth_e)
    
    for cyc in range(cycles):
        sum1= 0
        for ix in range(100):
            for iz in range(81):
                sum1 = sum1 + data[nvar-1,cyc,ix,iz]#/dataDensity[nvar-1,cyc,ix,iz]
                #AvgTemp[i] = np.mean(data[nvar-1,i,:,:])
        AvgTemp[cyc] = sum1 #np.mean(data[nvar-1,i,:,:])
    L = len(AvgTemp)%365
    HillslopeT['times'] = np.array(times)
    HillslopeT_Avg = np.array(AvgTemp)   
    HillslopeT['temp-daily_avg'] = HillslopeT_Avg
    HillslopeT['temp-annual_avg'] = np.reshape(HillslopeT_Avg[0:-L], (-1,365)).mean(axis=1)

    return HillslopeT


In [31]:
scenarios = ['B', 'S30', 'S60', 'S90']
permafrost = ['ALT', 'shallower', 'shallow', 'deep', 'deeper', 'deepest', 'lower']
depth1 = [[0,26],[26,46],[46,54],[54,64],[64,70],[70,75], [75,80]] # active layer, 0.5-2 m, 2-3.5m

In [32]:
# return water content in the layers/intervals
def get_water_content_layers(season = " ", start=0, end=-1,depth_s=-1,depth_e=-1):
    data = []
    dataDensity = []
    times = []
    if season == "B":
        data = Data_B[varnames[0]]
        dataDensity = Data_B[varnames[1]]
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30[varnames[0]]
        dataDensity = Data_S30[varnames[1]]
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60[varnames[0]]
        dataDensity = Data_S60[varnames[1]]
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90[varnames[0]]
        dataDensity = Data_S90[varnames[1]]
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeWC = dict()
    
    AvgTemp = np.zeros(cycles)
    MaxTemp = np.zeros(cycles)
    MinTemp = np.zeros(cycles)
    
    for cyc in range(cycles):
        sum1= 0
        for ix in range(100):
            for iz in range(81-depth_e,81-depth_s):
                
                sum1 = sum1 + data[nvar-1,cyc,ix,iz]#/dataDensity[nvar-1,cyc,ix,iz]
                
        AvgTemp[cyc] = sum1
        
    L = len(AvgTemp)%365
    HillslopeWC['times'] = np.array(times)
    HillslopeWC_Avg = np.array(AvgTemp)   
    HillslopeWC['temp-daily_avg'] = HillslopeWC_Avg
    HillslopeWC['temp-annual_avg'] = np.reshape(HillslopeWC_Avg[:-L], (-1,365)).mean(axis=1)

    return HillslopeWC

        

In [33]:
# for total water content in the system
Data_avg_WC = dict()
scenarios = ['B', 'S30', 'S60', 'S90']

for s in scenarios:
    d = get_water_content(s)
    Data_avg_WC[s + '_temp'] = d['temp-annual_avg']
    Data_avg_WC[s + 'daily_temp'] = d['temp-daily_avg']
    Data_avg_WC[s + '_time'] = d['times']

In [37]:
print (Data_avg_WC['Bdaily_temp'][-1]/55555.,Data_avg_WC['S90daily_temp'][-1]/55555., Data_avg_WC['Bdaily_temp'][-1]/55555. - Data_avg_WC['S90daily_temp'][-1]/55555.)

(3041.7200000753073, 3041.5805468520925, 0.1394532232147867)


In [35]:
# for water content in the layers
start1_,end1_, = 0,-1
Data_avg_WC_layer = dict()
scenarios = ['B', 'S30', 'S60', 'S90']
#permafrost = ['top', 'shallow', 'deep']
for i in range(len(permafrost)):
    for s in scenarios:
        d = get_water_content_layers(s,start=start1_,end=end1_,depth_s=depth1[i][0],depth_e=depth1[i][1])
        Data_avg_WC_layer[s + '_' + permafrost[i] + '_temp'] = d['temp-annual_avg']
        Data_avg_WC_layer[s + '_daily' + '_' + permafrost[i] + '_temp'] = d['temp-annual_avg']
        Data_avg_WC_layer[s + '_' + permafrost[i] +'_time'] = d['times']

In [ ]:
sumB = 0
sumS30 =0
sumS60 =0
sumS90 =0

K1 = ['ALT', 'shallower', 'shallow_', 'deep_', 'deeper_','deepest_','_lower'] 
for key in Data_avg_WC_layer.keys():
    K = K1[6]
    if 'B' in key and K in key and 'time' not in key:
        print (key, Data_avg_WC_layer[key][-1])
        sumB = sumB + Data_avg_WC_layer[key][-1]
        #print (key, sumB)
    if 'S30' in key and K in key and 'time' not in key:
        sumS30 = sumS30 + Data_avg_WC_layer[key][-1]
    if 'S60' in key and K in key and 'time' not in key:
        sumS60 = sumS60 + Data_avg_WC_layer[key][-1]
    if 'S90' in key and K in key and 'time' not in key:
        sumS90 = sumS90 + Data_avg_WC_layer[key][-1]
#3043, 
print ( np.round(sumB,4), np.round(sumS30,4), np.round(sumS60,4), np.round(sumS90,4))

In [ ]:
x1 = [32.2832, 32.2598, 32.2444, 32.1897]
x2 =  [81.9271, 81.9211, 81.9165, 81.9176]
x3 =  [83.9, 83.8954, 83.8892, 83.8772]
x4 =  [261.7341, 261.7226, 261.7068, 261.676]
x5 =  [376.4685, 376.4558, 376.4376, 376.4017]
x6 =  [584.2781, 584.2647, 584.2454, 584.2065]
x7 =  [1623.2563, 1623.2453, 1623.2294, 1623.1963]

In [ ]:
print (x1[0] - x1[-1])
print (x2[0] - x2[-1])
print (x3[0] - x3[-1])
print (x4[0] - x4[-1])
print (x5[0] - x5[-1])
print (x6[0] - x6[-1])
print (x7[0] - x7[-1])

In [ ]:
print (0.0935 + 0.0095 + 0.0228 + 0.0581 + 0.0668 + 0.0716 + 0.0599)

In [46]:
#end of simulation
sumB = 0
sumS30 =0
sumS60 =0
sumS90 =0

K1 = ['ALT', 'shallower', 'shallow_', 'deep_', 'deeper_','deepest_','_lower'] 
for key in Data_avg_WC_layer.keys():
    K = K1[6]
    if '_daily' in key and 'time' not in key:
        if 'B' in key and K in key:
            print (key, Data_avg_WC_layer[key][-1])
            sumB = sumB + Data_avg_WC_layer[key][-1]
            #print (key, sumB)
        if 'S30' in key and K in key:
            sumS30 = sumS30 + Data_avg_WC_layer[key][-1]
        if 'S60' in key and K in key:
            sumS60 = sumS60 + Data_avg_WC_layer[key][-1]
        if 'S90' in key and K in key:
            sumS90 = sumS90 + Data_avg_WC_layer[key][-1]
#3043, 
print ( np.round(sumB/55555.,4), np.round(sumS30/55555.,4), np.round(sumS60/55555.,4), np.round(sumS90/55555.,4))

('B_daily_lower_temp', 51300357.350473054)
(923.4157, 923.4157, 923.4157, 923.4157)


In [47]:
#x1 = [31.8613, 31.8312, 31.806, 31.7334]
#x2 = [ 76.2245, 76.221, 76.2209, 76.2299]
#x3 = [ 78.0564, 78.052, 78.0461, 78.0347]
#x4 = [ 233.6798, 233.6693, 233.6549, 233.6268]
#x5 = [ 336.0121, 336.0002, 335.9832, 335.9499]
#x6 = [ 521.0415, 521.0283, 521.0093, 520.9713]
#x7 = [ 924.4129, 924.4014, 924.3846, 924.3503]
x1 = [31.7985, 31.7713, 31.75, 31.685]
x2 = [76.0988, 76.1008, 76.1081, 76.1318]
x3 = [77.9434, 77.9434, 77.9434, 77.9434]
x4 = [233.354, 233.354, 233.354, 233.354]
x5 = [335.5556, 335.5556, 335.5556, 335.5556]
x6 = [520.38, 520.38, 520.38, 520.38]
x6 = [923.4157, 923.4157, 923.4157, 923.4157]
print (x1[0] - x1[-1])
print (x2[0] - x2[-1])
print (x3[0] - x3[-1])
print (x4[0] - x4[-1])
print (x5[0] - x5[-1])
print (x6[0] - x6[-1])
print (x1[0]+x2[0]+x3[0] +x4[0]+x5[0]+x6[0] +x7[0])
print (x1[3]+x2[3]+x3[3] +x4[3]+x5[3]+x6[3] +x7[3])

0.1135
-0.033
0.0
0.0
0.0
0.0
2602.5789
2602.4358


In [84]:
# Three zones : Active layer, shallow permafrost, deep permafrost
outfile='/Users/ajc/Desktop/maxsnow/'
def Plot_Wetlands_Uplands():
    fig, axs = plt.subplots(3,1, figsize=(6,6), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15*0
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = [r'S$_{0}$', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(311)
    X = x1

    ax1.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
    
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=10, fontweight='normal')

    ax1.set_ylabel('Water content [m$^3$]',fontsize=10, fontweight='normal')
    ax1.set_ylim((31.6, 31.82))
    ax1.text(0.5, -3.85, 'Active layer',fontsize=10, fontweight='normal')
    plt.tight_layout()
    
    # Up land
    ax1=plt.subplot(312)
    X = x2
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
    
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=10, fontweight='normal')

    #ax1.set_ylim((-7, 0))
    
    ax1.text(0.5, -6.85, 'Active layer',fontsize=10, fontweight='normal')
    plt.tight_layout()
    
    # --------------------------------- MANIMUM ---------------------------
    ax3=plt.subplot(313)
    X = x3
    ax3.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
       
    ax3.set_xticks(xpos)
    ax3.set_xticklabels(Xticks,fontsize=10, fontweight='normal')

    ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=10, fontweight='normal')
    
    ax3.set_ylim((-4, 0))
    ax3.text(0.5, -3.85, 'Shallower permafrost',fontsize=10, fontweight='normal')
    
    

    ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=10, fontweight='normal')
    ax3.set_xlabel('Numerical scenario',fontsize=11, fontweight='normal')
    ax3.set_ylim((-4, 0))
    ax3.text(0.5, -3.85, 'Shallow permafrost',fontsize=10, fontweight='normal')
    
    
    # --------------------------------- Annual time series ---------------------------

    ax3.set_ylim((-7, 0))    
    #ax3.legend(loc='lower right', fontsize=8,labelspacing=0.01,bbox_to_anchor=(0.9,-0.01, .1, .1))
    #plt.tight_layout()
    
    
    #plt.savefig(outfile+'avg-temp-3zones.pdf', bbox_inches='tight', dpi=100)
    

In [85]:
Plot_Wetlands_Uplands()

In [98]:
# Three zones : Active layer, shallow permafrost, deep permafrost
outfile='/Users/ajc/Desktop/maxsnow/'
def plot_water_content():
    fig, axs = plt.subplots(1,1, figsize=(4,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    color=['k', 'g','b','r']
    
    
    plt.tight_layout()
    #axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = [r'S$_{0}$', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(111)
    X = x1#[Data_avg_WC[ 'B_temp'][-1], Data_avg_WC[ 'S30_temp'][-1], Data_avg_WC[ 'S60_temp'][-1],Data_avg_WC[ 'S90_temp'][-1]]
    
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
       
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=10, fontweight='normal')
    ax1.set_title('Active layer water content',fontsize=12, fontweight='normal')
    ax1.set_ylabel('Water content [m$^3$]',fontsize=12, fontweight='normal')
    ax1.set_xlabel('Numerical scenario',fontsize=12, fontweight='normal')
    ax1.set_ylim((31.6, 31.85))    
     
        
    plt.tight_layout()
    
    plt.savefig(outfile+'water_budget-ALT.pdf', bbox_inches='tight', dpi=100)
    

In [99]:
plot_water_content()

In [ ]:
d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
d2=d['surface-molar_density_liquid.cell.0']

c1 = sorted(d1.keys(),key=int)
print (len(d['surface-water_content.cell.0'][c1[-1]]))
#print (list(d2[c1[-1]]))
sum1 = 0
for ds,dd  in zip(d['surface-water_content.cell.0'][c1[-1]], d['surface-molar_density_liquid.cell.0'][c1[-1]]):
    sum1 = sum1 + ds/dd
surf_B = sum1
print ('B', surf_B)
#surf_B=sum(d['surface-water_content.cell.0'][c1[-1]])/55555.
#print ('B', surf_B)

d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S30_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
c1 = sorted(d1.keys(),key=int)
#print (c1[-1])
sum1 = 0
for ds,dd  in zip(d['surface-water_content.cell.0'][c1[-1]], d['surface-molar_density_liquid.cell.0'][c1[-1]]):
    sum1 = sum1 + ds/dd
    
surf_S30=sum1#sum(d['surface-water_content.cell.0'][c1[-1]])/55555.
print ('S30', surf_S30)

d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S60_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
c1 = sorted(d1.keys(),key=int)

sum1 = 0
for ds,dd  in zip(d['surface-water_content.cell.0'][c1[-1]], d['surface-molar_density_liquid.cell.0'][c1[-1]]):
    sum1 = sum1 + ds/dd
surf_S60=sum1 #sum(d['surface-water_content.cell.0'][c1[-1]])/55555.
print ('S60 ', surf_S60)

d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
c1 = sorted(d1.keys(),key=int)

sum1=0
for ds,dd  in zip(d['surface-water_content.cell.0'][c1[-1]], d['surface-molar_density_liquid.cell.0'][c1[-1]]):
    sum1 = sum1 + ds/dd
surf_S90=sum1 #sum(d['surface-water_content.cell.0'][c1[-1]])/55555.
print ('S90', surf_S90)

#print (11.7 + Data_avg_WC[ 'B_temp'][-1]) # 11.7 is discharge
#print (12. + Data_avg_WC[ 'S30_temp'][-1])
#print (12.125 + Data_avg_WC[ 'S60_temp'][-1])
#print (12.25 + Data_avg_WC[ 'S90_temp'][-1])

print ("----------")
print (Data_avg_WC['B_temp'][-1])
print (Data_avg_WC['S30_temp'][-1])
print (Data_avg_WC['S60_temp'][-1])
print (Data_avg_WC['S90_temp'][-1])

In [ ]:
WC_surf = [surf_B,surf_S30,surf_S60,surf_S90]
print (WC_surf)

In [ ]:

#Evaporation
d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/visdump_surface_data.h5','r')
key_evap = 'surface-evaporative_flux.cell.0'
key_sm = 'surface-snowmelt.cell.0'
d1=d[key_evap]


c1 = sorted(d1.keys(),key=int)
#print (c1[-365],c1[-1])
surf_B = []
surf_B_snow = []

for i in c1[-365:]:
    #print (i)
    ds = d[key_evap][i]
    sum1 = np.sum(ds)*86400 # Area = 1 x 1 m^2
    surf_B.append(sum1)

    #ds = d[key_sm][c1[-i]]
    #sum1 = np.sum(ds)*86400 # Area = 1 x 1 m^2
    #surf_B_snow.append(sum1)
    
#print ('B', surf_B)

d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S30_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
c1 = sorted(d1.keys(),key=int)

surf_S30 = []
surf_S30_snow = []
for i in c1[-365:]:
    #print (i)
    ds = d[key_evap][i]
    sum1 = np.sum(ds)*86400
    surf_S30.append(sum1)

    #ds = d[key_sm][c1[-i]]
    #sum1 = np.sum(ds)*86400 # Area = 1 x 1 m^2
    #surf_S30_snow.append(sum1)
    
d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S60_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
c1 = sorted(d1.keys(),key=int)

surf_S60 = []
surf_S60_snow = []
for i in c1[-365:]:
    ds = d[key_evap][i]
    sum1 = np.sum(ds)*86400
    surf_S60.append(sum1)

    #ds = d[key_sm][c1[-i]]
    #sum1 = np.sum(ds)*86400 # Area = 1 x 1 m^2
    #surf_S60_snow.append(sum1)
    
d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
c1 = sorted(d1.keys(),key=int)

surf_S90 = []
surf_S90_snow = []
for i in c1[-365:]:
    ds = d[key_evap][i]
    sum1 = np.sum(ds)*86400
    surf_S90.append(sum1)
    
    #ds = d[key_sm][c1[-i]]
    #sum1 = np.sum(ds)*86400 # Area = 1 x 1 m^2
    #surf_S90_snow.append(sum1)

fig,axs = plt.subplots(1,1, figsize=(5,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
ax1 = plt.subplot('111')
plt.plot(surf_B,'k',label=r'S$_{0}$')
plt.plot(surf_S30,'g',label=r'S$_{30}$')
plt.plot(surf_S60,'b',label=r'S$_{60}$')
plt.plot(surf_S90,'r',label=r'S$_{90}$')
ax1.set_ylabel('Evaporative flux [m$^3$]')
ax1.set_xlabel('Time [day]')
ax1.set_xlim([0,366])
ax1.legend(loc='upper left', fontsize=9,labelspacing=0.1,bbox_to_anchor=(0.01, 0.9, .1, .1))


plt.savefig(outfile+'/evaporative_flux.pdf', bbox_inches='tight', dpi=100)
plt.show()



#print (B_E[-1], S30_E[-1], S60_E[-1],S90_E[-1])
print (np.sum(surf_B),np.sum(surf_S30),np.sum(surf_S60),np.sum(surf_S90))
#print (np.sum(surf_B_snow),np.sum(surf_S30_snow),np.sum(surf_S60_snow),np.sum(surf_S90_snow))
x11 = np.cumsum(surf_B) 
x21 = np.cumsum(surf_S30)
x31 = np.cumsum(surf_S60)
x41 = np.cumsum(surf_S90)

#print (surf_B[90:120])

In [ ]:
outfile='/Users/ajc/Desktop/maxsnow/'
%matplotlib tk
fig,axs = plt.subplots(1,1, figsize=(5,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()

end = 7300-264
l1,l2 = -365-1, -1

B = np.cumsum(surf_B) 
S30 = np.cumsum(surf_S30)
S60 = np.cumsum(surf_S60)
S90 = np.cumsum(surf_S90)

print (B[-1],S30[-1],S60[-1],S90[-1])
#print (S90[-1]- B[-1], DataSim['B_time'][l1:l1+3])
#print (B_trapz, S30_trapz, S60_trapz,S90_trapz)
print ('----------------')
ax1 = plt.subplot('111')

plt.plot(B,'k',label=r'S$_{0}$')
plt.plot(S30,'g',label=r'S$_{30}$')
plt.plot(S60,'b',label=r'S$_{60}$')
plt.plot(S90,'r',label=r'S$_{90}$')
ax1.set_ylabel('Cumulative evaporation [m$^3$]')
ax1.set_xlabel('Time [day]')
ax1.set_xlim([0,366])
ax1.legend(loc='upper left', fontsize=9,labelspacing=0.1,bbox_to_anchor=(0.01, 0.9, .1, .1))

# inset axes....
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

axins1 = zoomed_inset_axes(ax1, 9.5, loc='center',bbox_to_anchor=(260, 60, 60., 60.))
axins1.plot(B,'k')
axins1.plot(S30,'g')
axins1.plot(S60,'b')
axins1.plot(S90,'r')
axins1.set_xlim(354, 364)
#axins1.set_ylim(134, 144)
axins1.set_ylim(20.5, 21.5)

mark_inset(ax1, axins1, loc1=2, loc2=4, fc="none", ec="0.5")

plt.savefig(outfile+'/evaporative_cumm.pdf', bbox_inches='tight', dpi=100)
plt.show()


In [ ]:
print (11.69+B_E[-1] + x1[0] + x2[0] + WC_surf[0], 
       11.98 + S30_E[-1] + x1[1] + x2[1] + WC_surf[1],  
       12.14 + S60_E[-1] + x1[2] + x2[2] + WC_surf[2], 
       12.27 + S90_E[-1] + x1[3] + x2[3] + WC_surf[3])

In [ ]:
infileB=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/visdump_data.h5','r')
infileS90=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/visdump_data.h5','r')


In [ ]:
DataBB = infileB['water_content.cell.0']
DataS90 = infileS90['water_content.cell.0']

In [ ]:
cyclesB = sorted(DataBB.keys(), key=int)
cyclesS90 = sorted(DataS90.keys(), key=int)

In [ ]:
d1 = np.array(DataBB[cyclesB[0]])
d2 = np.array(DataS90[cyclesS90[0]])

In [ ]:
plt.plot(d1,'r')
#plt.plot(d2,'k')
plt.show()

In [ ]:
print (d1[10],np.sum(d1),np.sum(d2))

In [ ]:
d3 = Data_B[varnames[0]]
d4 = Data_S90[varnames[0]]

In [ ]:
Bx = []
S90x = []
for k in range(365):
    x1 = 0
    x2 = 0
    for i in range(100):
        for j in range(81):
            x1 = x1 + d3[2,k,i,j]
            x2 = x2 + d4[2,k,i,j]
            if (x1 < 0 or x2 <0):
                print (k,i,j,x1,x2)
                break
    Bx.append(x1/55555.)
    S90x.append(x2/55555.)
#print (len(d3[2,0,:,:]),x/55555. , np.sum(d1)/55555.)

In [ ]:
print (Data_avg_WC.keys())
print (np.sum(Data_avg_WC['Bdaily_temp'][:365]),np.sum(Data_avg_WC['S90daily_temp'][0:365]))

In [ ]:
plt.plot(Data_avg_WC['Bdaily_temp'][:365],'k')
plt.plot(Data_avg_WC['S90daily_temp'][:365],'r')
plt.plot(Bx,'g')
plt.plot(S90x,'m')

In [ ]:
d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
d2=d['surface-molar_density_liquid.cell.0']

c1 = sorted(d1.keys(),key=int)

surf_B = []
for i in range(365):
    sum1 = 0
    for ds,dd  in zip(d['surface-water_content.cell.0'][c1[i]], d['surface-molar_density_liquid.cell.0'][c1[i]]):
        sum1 = sum1 + ds/dd
    surf_B.append(sum1)


d=h5py.File('/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/visdump_surface_data.h5','r')
d1=d['surface-water_content.cell.0']
c1 = sorted(d1.keys(),key=int)

surf_S90 = []
for i in range(365):
    sum1 = 0
    for ds,dd  in zip(d['surface-water_content.cell.0'][c1[i]], d['surface-molar_density_liquid.cell.0'][c1[i]]):
        sum1 = sum1 + ds/dd
    surf_S90.append(sum1)


In [ ]:
IntegB = [x+y for x,y in zip(Data_avg_WC['Bdaily_temp'][:365],surf_B)]
IntegS90 = [x+y for x,y in zip(Data_avg_WC['S90daily_temp'][:365],surf_S90)]
plt.plot(Data_avg_WC['Bdaily_temp'][:365],'k')
plt.plot(Data_avg_WC['S90daily_temp'][:365],'r')
plt.plot(Bx,'g')
plt.plot(S90x,'m')
plt.plot(IntegB,'b')
plt.plot(IntegS90,'c')